In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from crewai import LLM

llm = LLM(model="gemini/gemini-2.0-flash",
          temperature=0.2
        )

llm.call("What are the main causes of the recent coronavirus pandemic?")

"The COVID-19 pandemic was caused by the Severe Acute Respiratory Syndrome Coronavirus 2 (SARS-CoV-2). Here's a breakdown of the main causes:\n\n*   **The Virus: SARS-CoV-2:** This is the direct cause. It's a novel coronavirus, meaning it was a new strain that hadn't been seen in humans before.\n\n*   **Zoonotic Spillover:** The virus is believed to have originated in bats. It then likely jumped to an intermediate animal host (the exact animal is still debated, but possibilities include pangolins or other wildlife sold in markets) before infecting humans. This jump from animals to humans is called zoonotic spillover.\n\n*   **Wet Markets/Wildlife Trade:** The Huanan Seafood Wholesale Market in Wuhan, China, was initially identified as a potential epicenter. These markets often sell live and dead animals, creating opportunities for viruses to jump between species and to humans. The close proximity of humans and animals in these environments increases the risk of spillover.\n\n*   **Huma

### Creating email assistant using crewai

In [10]:
original_email = """
Hey team,

Just wanted to circle back regarding our recent sync-up. With a strategic focus on the downstream impact of the coronavirus on our team’s 
overall KPIs (Key Performance Indicators) and productivity metrics, here are some key takeaways:

1. The virus has affected {num_employees} FTEs (Full-Time Equivalents) globally.
2. We’ve rolled out new SOPs (Standard Operating Procedures) to ensure employee safety and business continuity.
3. Our productivity delta has improved by {productivity_increase}% through optimized bandwidth allocation and reducing non-value-add 
touchpoints (calls and meetings).
4. We’ve also addressed concerns around employee engagement, morale, and overall EQ (Emotional Quotient) within the team.

Hope this intel helps you make data-driven decisions and sustain a high-performance culture.
"""


### Jargon replacement tool
from crewai.tools import BaseTool

class JargonReplacementTool(BaseTool):
    name: str = "Jargon Replacement Tool"
    description: str = "A tool that can be used to replace jargon in a text with more professional terms."
    
    def _run(self, email: str) -> str:
        
        # Replace jargon with professional terms using a dictionary
        replacements = {
            "circle back": "follow up or revisit",
            "sync-up": "synchronization meeting",
            "downstream impact": "consequences on subsequent areas",
            "KPIs": "Key Performance Indicators",
            "takeaways": "key conclusions or summary points",
            "FTEs": "Full-Time Equivalents",
            "SOPs": "Standard Operating Procedures",
            "business continuity": "ongoing operations during disruptions",
            "delta": "change or difference",
            "bandwidth allocation": "distribution of available work capacity",
            "non-value-add touchpoints": "unnecessary interactions",
            "intel": "information or insight",
            "data-driven": "based on measurable evidence",
            "high-performance culture": "efficient and motivated workplace",
            "EQ": "Emotional Quotient or emotional intelligence"
        }
        
        suggestions = []
        for jargon, replacement in replacements.items():
            if jargon.lower() in email.lower():
                suggestions.append(f"{jargon} should be replaced with {replacement}.")
                
        return f"Jargon Replacement Suggestions:\n{'\n'.join(suggestions)}" if suggestions else "No jargon found."
    
jt = JargonReplacementTool()
print(jt.run(original_email))


Using Tool: Jargon Replacement Tool
Jargon Replacement Suggestions:
circle back should be replaced with follow up or revisit.
sync-up should be replaced with synchronization meeting.
downstream impact should be replaced with consequences on subsequent areas.
KPIs should be replaced with Key Performance Indicators.
takeaways should be replaced with key conclusions or summary points.
FTEs should be replaced with Full-Time Equivalents.
SOPs should be replaced with Standard Operating Procedures.
business continuity should be replaced with ongoing operations during disruptions.
delta should be replaced with change or difference.
bandwidth allocation should be replaced with distribution of available work capacity.
intel should be replaced with information or insight.
data-driven should be replaced with based on measurable evidence.
high-performance culture should be replaced with efficient and motivated workplace.
EQ should be replaced with Emotional Quotient or emotional intelligence.


In [ ]:
# Create Email Assistant Agent
from crewai import Agent, Task, Crew

email_assistant = Agent(
    role="Email Assistant",
    goal="Improve emails by providing relevant information, formatting, and maintaining a consistent tone.",
    backstory="You are an expert email assistant skilled in profession email writing and formatting. You have been trained on various email templates and best practices.",
    llm=llm,
    tools=[jt],
    verbose=True,
)

In [12]:
improve_email_task = Task(
    description=f"Improve the following email into a professional email: {original_email}",
    expected_output="A professional and well-formatted email",
    agent=email_assistant,
)

In [13]:
crew = Crew(agents=[email_assistant], tasks=[improve_email_task], verbose=True)
result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 949bb512-1549-418f-a708-13fa46c940a1                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant                                                                                         │
│                                                                                                                 │
│  Task: Improve the following email into a professional email:                                                   │
│  Hey team,                                                                                                      │
│                                                                                                                 │
│  Just wanted to circle back regarding our recent sync-up. With a strategic focus on the downstream impact of    │
│  the coronavirus on our team’s                                                                                  │
│  overall KPIs (Key Performance Indicators) and productivity metrics, here are some key takeaways:               │
│                                                                                                                 │
│  1. The virus has affected {num_employees} FTEs (Full-Time Equivalents) globally.                               │
│  2. We’ve rolled out new SOPs (Standard Operating Procedures) to ensure employee safety and business            │
│  continuity.                                                                                                    │
│  3. Our productivity delta has improved by {productivity_increase}% through optimized bandwidth allocation and  │
│  reducing non-value-add                                                                                         │
│  touchpoints (calls and meetings).                                                                              │
│  4. We’ve also addressed concerns around employee engagement, morale, and overall EQ (Emotional Quotient)       │
│  within the team.                                                                                               │
│                                                                                                                 │
│  Hope this intel helps you make data-driven decisions and sustain a high-performance culture.                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant                                                                                         │
│                                                                                                                 │
│  Thought: I need to refine the email to make it more professional by replacing jargon and ensuring clarity. I   │
│  will use the Jargon Replacement Tool to identify and replace any jargon with more appropriate terms.           │
│                                                                                                                 │
│  Using Tool: Jargon Replacement Tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"email\": \"Hey team,\\n\\nJust wanted to circle back regarding our recent sync-up. With a strategic focus  │
│  on the downstream impact of the coronavirus on our team\\u2019s \\noverall KPIs (Key Performance Indicators)   │
│  and productivity metrics, here are some key takeaways:\\n\\n1. The virus has affected {num_employees} FTEs     │
│  (Full-Time Equivalents) globally.\\n2. We\\u2019ve rolled out new SOPs (Standard Operating Procedures) to      │
│  ensure employee safety and business continuity.\\n3. Our productivity delta has improved by                    │
│  {productivity_increase}% through optimized bandwidth allocation and reducing non-value-add \\ntouchpoints      │
│  (calls and meetings).\\n4. We\\u2019ve also addressed concerns around employee engagement, morale, and         │
│  overall EQ (Emotional Quotient) within the team.\\n\\nHope this intel helps you make data-driven decisions     │
│  and sustain a high-performance culture.\"}"                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Follow-up on Recent Synchronization Meeting                                                           │
│                                                                                                                 │
│  Dear Team,                                                                                                     │
│                                                                                                                 │
│  Following up on our recent synchronization meeting, and with a strategic focus on the consequences of the      │
│  coronavirus on our team’s overall Key Performance Indicators and productivity metrics, here are some key       │
│  conclusions:                                                                                                   │
│                                                                                                                 │
│  1.  The virus has affected {num_employees} Full-Time Equivalents globally.                                     │
│  2.  We’ve rolled out new Standard Operating Procedures to ensure employee safety and ongoing operations        │
│  during disruptions.                                                                                            │
│  3.  Our productivity change has improved by {productivity_increase}% through optimized distribution of         │
│  available work capacity and reducing unproductive interactions (calls and meetings).                           │
│  4.  We’ve also addressed concerns around employee engagement, morale, and overall Emotional Quotient within    │
│  the team.                                                                                                      │
│                                                                                                                 │
│  I hope this information helps you make decisions based on measurable evidence and sustain an efficient and     │
│  motivated workplace.                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2c2c3a9f-d537-4833-983d-325bc5eefc37                                                                     │
│  Agent: Email Assistant                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 949bb512-1549-418f-a708-13fa46c940a1                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Follow-up on Recent Synchronization Meeting                                             │
│                                                                                                                 │
│  Dear Team,                                                                                                     │
│                                                                                                                 │
│  Following up on our recent synchronization meeting, and with a strategic focus on the consequences of the      │
│  coronavirus on our team’s overall Key Performance Indicators and productivity metrics, here are some key       │
│  conclusions:                                                                                                   │
│                                                                                                                 │
│  1.  The virus has affected {num_employees} Full-Time Equivalents globally.                                     │
│  2.  We’ve rolled out new Standard Operating Procedures to ensure employee safety and ongoing operations        │
│  during disruptions.                                                                                            │
│  3.  Our productivity change has improved by {productivity_increase}% through optimized distribution of         │
│  available work capacity and reducing unproductive interactions (calls and meetings).                           │
│  4.  We’ve also addressed concerns around employee engagement, morale, and overall Emotional Quotient within    │
│  the team.                                                                                                      │
│                                                                                                                 │
│  I hope this information helps you make decisions based on measurable evidence and sustain an efficient and     │
│  motivated workplace.                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Subject: Follow-up on Recent Synchronization Meeting

Dear Team,

Following up on our recent synchronization meeting, and with a strategic focus on the consequences of the coronavirus on our team’s overall Key Performance Indicators and productivity metrics, here are some key conclusions:

1.  The virus has affected {num_employees} Full-Time Equivalents globally.
2.  We’ve rolled out new Standard Operating Procedures to ensure employee safety and ongoing operations during disruptions.
3.  Our productivity change has improved by {productivity_increase}% through optimized distribution of available work capacity and reducing unproductive interactions (calls and meetings).
4.  We’ve also addressed concerns around employee engagement, morale, and overall Emotional Quotient within the team.

I hope this information helps you make decisions based on measurable evidence and sustain an efficient and motivated workplace.
